# Importing packages

In [1]:
import pandas as pd
import numpy as np
import math
import random

# Initializing the node

In [2]:
class Node:
    """Contains the information of the node and another nodes of the Decision Tree."""

    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None


# Defining the model

In [3]:
class DecisionTreeClassifier:
    
    def __init__(self,X,feature_names,labels):
        self.X=X
        self.feature_names=feature_names
        self.labels=labels
        self.labelCategories = list(set(labels))
        self.labelCategoriesCount = [list(labels).count(x) for x in self.labelCategories]
        self.node = None
        self.entropy = self._get_entropy([x for x in range(len(self.labels))])
    
    def _get_entropy(self, x_ids):
        
        labels = [self.labels[i] for i in x_ids]
      
        label_count = [labels.count(x) for x in self.labelCategories]
        entropy = sum([-count / len(x_ids) * math.log(count / len(x_ids), 2)
                       if count else 0
                       for count in label_count
                      ])
        
        return entropy
    
    def _get_information_gain(self, x_ids, feature_id):
        
        # calculate total entropy
        info_gain = self._get_entropy(x_ids)
        # store in a list all the values of the chosen feature
        x_features = [self.X[x][feature_id] for x in x_ids]
        # get unique values
        feature_vals = list(set(x_features))
        # get frequency of each value
        feature_v_count = [x_features.count(x) for x in feature_vals]
        # get the feature values ids
        feature_v_id = [
            [x_ids[i]
            for i, x in enumerate(x_features)
            if x == y]
            for y in feature_vals
        ]

        # compute the information gain with the chosen feature
        info_gain_feature = sum([v_counts / len(x_ids) * self._get_entropy(v_ids)
                            for v_counts, v_ids in zip(feature_v_count, feature_v_id)])

        info_gain = info_gain - info_gain_feature

        return info_gain
    
    def _get_feature_max_information_gain(self, x_ids, feature_ids):
        
        # get the entropy for each feature
        features_entropy = [self._get_information_gain(x_ids, feature_id) for feature_id in feature_ids]
        # find the feature that maximises the information gain
        max_id = feature_ids[features_entropy.index(max(features_entropy))]
        return self.feature_names[max_id], max_id
    

    def printtree(self):
        curr = self.node
        st = []
        while curr:
            print("value",curr.value)
            if curr.next:
                print("appending next node")
                st.append(curr.next)
            else:
                print("No Next Node")
            if curr.childs:
                print("appending " , len(curr.childs)," children node")
                st.append(curr.childs[0])
            else:
                print("No children")
            if len(st) > 0:
                curr = st.pop(0)
            else:
                break
                
    
    def id3(self):

        x_ids = [x for x in range(len(self.X))]
        # assign an unique number to each featuer
        feature_ids = [x for x in range(len(self.feature_names))]
        # define node variable - instance of the class Node
        self.node = self._id3_recv(x_ids, feature_ids, self.node)                
    
    def get_prediction(self,record):
        curr = self.node
        check = 0
        while curr:
            check = check + 1
            if check > 100:
                break
            if curr.value in self.feature_names:
                feature_index = self.feature_names.index(curr.value)
            else:
                return curr.value
            for i in curr.childs:
                if i.next:
                    if i.value == record[feature_index]:
                        curr = i.next
                else:
                    return i.value


    def _id3_recv(self, x_ids, feature_ids, node):
        
        if not node:
            node = Node()  # initialize nodes
        # sorted labels by instance id
        labels_in_features = [self.labels[x] for x in x_ids]
        # print(set(labels_in_features))
        # if all the example have the same class (pure node), return node
        if len(set(labels_in_features)) == 1:
            node.value = self.labels[x_ids[0]]
            return node
        # if there are not more feature to compute, return node with the most probable class
        if len(feature_ids) == 0:
            node.value = max(set(labels_in_features), key=labels_in_features.count)  # compute mode
            return node
        # else...
        # choose the feature that maximizes the information gain
        best_feature_name, best_feature_id = self._get_feature_max_information_gain(x_ids, feature_ids)
        
        node.value = best_feature_name
        node.childs = []
        # value of the chosen feature for each instance
        feature_values = list(set([self.X[x][best_feature_id] for x in x_ids]))
       
        # loop through all the values
        for value in feature_values:
            child = Node()
            child.value = value  # add a branch from the node to each feature value in our feature
            node.childs.append(child)  # append new child node to current node
            child_x_ids = [x for x in x_ids if self.X[x][best_feature_id] == value]
            if not child_x_ids:
                child.next = max(set(labels_in_features), key=labels_in_features.count)
                print('')
            else:
                if feature_ids and best_feature_id in feature_ids:
                    to_remove = feature_ids.index(best_feature_id)
                    feature_ids.pop(to_remove)
                # recursively call the algorithm
                child.next = self._id3_recv(child_x_ids, feature_ids, child.next)

        
        return node


# Features and Labels

In [4]:
features=['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian','traveltime','studytime','failures','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic','famrel','freetime','goout','Dalc','Walc','health','G1','G2','G3']
todrop = ['absences']
data=pd.read_csv('student-por.csv')
data.drop(todrop,axis=1)
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [5]:
data=data.to_numpy()
labels=data[:,-1]
labels

array([11, 11, 12, 14, 13, 13, 13, 13, 17, 13, 14, 13, 12, 13, 15, 17, 14,
       14, 7, 12, 14, 12, 14, 10, 10, 12, 12, 11, 13, 12, 11, 15, 15, 12,
       12, 11, 14, 13, 12, 12, 10, 11, 15, 10, 11, 11, 13, 17, 13, 12, 13,
       16, 9, 12, 13, 12, 15, 16, 14, 16, 16, 16, 10, 13, 12, 16, 12, 10,
       11, 15, 11, 10, 11, 14, 11, 11, 11, 13, 10, 11, 12, 9, 11, 13, 12,
       12, 11, 15, 11, 10, 11, 13, 12, 14, 12, 13, 11, 12, 13, 13, 8, 16,
       12, 10, 16, 10, 10, 14, 11, 14, 14, 11, 10, 18, 10, 14, 16, 15, 11,
       14, 14, 13, 13, 13, 11, 9, 11, 11, 15, 13, 12, 8, 11, 13, 12, 14,
       11, 11, 11, 15, 10, 13, 12, 11, 11, 10, 10, 14, 9, 11, 9, 13, 11,
       13, 11, 6, 12, 10, 11, 13, 11, 8, 11, 0, 10, 13, 11, 13, 8, 10, 11,
       11, 1, 10, 9, 8, 10, 8, 8, 8, 11, 18, 13, 17, 10, 18, 10, 13, 15,
       11, 14, 10, 11, 13, 11, 13, 17, 14, 16, 14, 11, 16, 14, 10, 13, 12,
       12, 10, 12, 16, 14, 12, 16, 11, 15, 12, 15, 13, 13, 8, 12, 15, 13,
       12, 12, 12, 13, 11, 11, 15, 1

In [6]:
attributes=features[:-1]
attributes

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'G1',
 'G2']

In [7]:
for i in range(len(data)):
    for j in range(len(data[0])-1,len(data[0])-4,-1):
        if data[i][j]>=0 and data[i][j]<=5:
            data[i][j]=0
        elif data[i][j]>=6 and data[i][j]<=10:
            data[i][j]=1
        elif data[i][j]>=11 and data[i][j]<=15:
            data[i][j]=2
        else:
            data[i][j]=3

In [8]:
print(data)

[['GP' 'F' 18 ... 0 2 2]
 ['GP' 'F' 17 ... 1 2 2]
 ['GP' 'F' 15 ... 2 2 2]
 ...
 ['MS' 'F' 18 ... 2 2 1]
 ['MS' 'M' 17 ... 1 1 1]
 ['MS' 'M' 18 ... 1 2 2]]


# Bootstrap Sampling Method

In [9]:
def get_bootstrap_samples():
    train = []
    test=[]
    for i in range(int(2*(len(data)/3))):
        ind = random.randint(0,len(data)-1)
        train.append(data[ind])
    for i in range(int(len(data)/3)):
        ind = random.randint(0,len(data)-1)
        test.append(data[ind])
    return np.array(train),np.array(test)

In [10]:
train_set,test_set = get_bootstrap_samples()
# print(train_set)

In [11]:
obj=DecisionTreeClassifier(train_set,attributes,labels)
obj.id3()

# Predicting Samples

In [12]:
def predict(test):
    count = 0
    conf_matrix = [[0 for x in range(5)] for y in range(5)]
    for i in test:
        actual_val = i[-1]
        predicted_val = obj.get_prediction(i)
        if not predicted_val:
            predicted_val=1
        conf_matrix[actual_val][predicted_val] = conf_matrix[actual_val][predicted_val] + 1
        conf_matrix[actual_val][4] = conf_matrix[actual_val][4] + 1
        conf_matrix[4][predicted_val] = conf_matrix[4][predicted_val] + 1
        if actual_val == predicted_val:
            count = count + 1
  
        majority_class = None
        max_val = 0
        for i in range(4):
            if conf_matrix[i][4] > max_val:
                majority_class = i
                max_val = conf_matrix[i][4]

                total_true_positives = 0
                false_positives = len(test) - max_val
                false_negatives = 0

        for i in range(4):
            if i==majority_class:
                continue
            else:
                false_negatives = false_negatives + conf_matrix[majority_class][i]
        for i in range(4):
            if i == majority_class:
                continue
            else:
                total_true_positives = total_true_positives + conf_matrix[i][i]

    accuracy = count/len(test)
    precision = total_true_positives/(len(test)-max_val) #total_true_positives + total_false_positives = length of test set without true negatives and false negatives
    recall = total_true_positives/(total_true_positives + false_negatives)
    fmeasure = (2 * precision * recall) / (precision + recall)
    return conf_matrix,accuracy,precision,recall,fmeasure

# Accuracy, precision, Recall, F Measure by Boostrap Sampling Method 

In [13]:
conf_matrix,accuracy,precision,recall,fmeasure = predict(test_set)

print("Confusion Matrix : ",conf_matrix)
print("Accuracy% : ", accuracy*100)
print("Precision% : ", precision*100)
print("Recall% : ", recall*100)
print("F Measure : ", fmeasure*100)

Confusion Matrix :  [[0, 1, 2, 0, 3], [0, 1, 61, 0, 62], [0, 5, 121, 0, 126], [0, 1, 24, 0, 25], [0, 8, 208, 0, 0]]
Accuracy% :  56.481481481481474
Precision% :  1.1111111111111112
Recall% :  16.666666666666664
F Measure :  2.0833333333333335


In [14]:
def predict1(test):
    count = 0
    none_count=0
    for i in test:
        out=obj.get_prediction(i)
       
        if i[-1] == out:
            count = count + 1
        if out==None:
            none_count+=1
    return (count/(len(test)-none_count))*100

# Accuracy by K-Fold Cross Validation

In [15]:
from numpy import array
from sklearn.model_selection import KFold
kfolds = KFold(10,shuffle = True)
validation_accuracy, test_accuracy =  [], []

for train,validation in kfolds.split(train_set):
    obj1=DecisionTreeClassifier(data[train],attributes,labels)
    obj1.id3()
    k_fold_validation_accuracy= predict1(data[validation])
    k_fold_test_accuracy= predict1(test_set)
    validation_accuracy.append(k_fold_validation_accuracy)
    test_accuracy.append(k_fold_test_accuracy)
validation_mean_accuracy = sum(validation_accuracy)//len(validation_accuracy)
test_mean_accuracy = sum(test_accuracy)//len(test_accuracy)

print(f'Mean Accuracy on overall Validation sets: {validation_mean_accuracy} %')
print(f'Mean Accuracy on test set: {test_mean_accuracy} %')

Mean Accuracy on overall Validation sets: 65.0 %
Mean Accuracy on test set: 57.0 %
